# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create  dataframe
city_df = pd.read_csv("../output_data/cities.csv")


city_df

,City,Cloudiness,Date,Humidity,Lat,Lon,Max Temp,Windspeed
0,Sobolevo,80,1627342911,77,54.4299,31.8996,62.94,5.48
1,Punta Arenas,0,1627342827,75,-53.1500,-70.9167,42.91,8.05
2,Albany,0,1627342680,74,42.6001,-73.9662,81.25,7.65
3,Baykit,27,1627342912,54,61.6700,96.3700,75.13,3.65
4,Saskylakh,70,1627342875,68,71.9167,114.0833,54.63,7.09
...,...,...,...,...,...,...,...,...
553,Tadpatri,85,1627343130,74,14.9167,78.0167,77.34,17.40
554,Hambantota,97,1627342871,82,6.1241,81.1185,79.90,16.91
555,Saint Anthony,1,1627342866,47,45.0205,-93.2180,89.74,10.36
556,Mossendjo,62,1627343131,92,-2.9506,12.7261,64.78,3.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
locations = city_df[["Lat", "Lon"]].astype(float)
Humidity = city_df["Humidity"].astype(float)
fig = gmaps.figure(map_type="TERRAIN")

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

new_city_df= pd.DataFrame(city_df, columns = ["City","Cloudiness","Windspeed","Max Temp"])



max_temp = (new_city_df["Max Temp"] < 80) & (new_city_df["Max Temp"] > 70)
wind_speed = new_city_df["Windspeed"] < 10
cloudiness = new_city_df["Cloudiness"] == 0

new_city_df

,City,Cloudiness,Windspeed,Max Temp
0,Sobolevo,80,5.48,62.94
1,Punta Arenas,0,8.05,42.91
2,Albany,0,7.65,81.25
3,Baykit,27,3.65,75.13
4,Saskylakh,70,7.09,54.63
...,...,...,...,...
553,Tadpatri,85,17.40,77.34
554,Hambantota,97,16.91,79.90
555,Saint Anthony,1,10.36,89.74
556,Mossendjo,62,3.11,64.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = pd.DataFrame(city_df, columns=["City", "Country", "Lat", "Lon"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lon,Hotel Name
0,Sobolevo,NaN,54.4299,31.8996,
1,Punta Arenas,NaN,-53.1500,-70.9167,
2,Albany,NaN,42.6001,-73.9662,
3,Baykit,NaN,61.6700,96.3700,
4,Saskylakh,NaN,71.9167,114.0833,
...,...,...,...,...,...
553,Tadpatri,NaN,14.9167,78.0167,
554,Hambantota,NaN,6.1241,81.1185,
555,Saint Anthony,NaN,45.0205,-93.2180,
556,Mossendjo,NaN,-2.9506,12.7261,


In [16]:
# find the closest  of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
          "type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key

}
# use iterrows to iterate through pandas dataframe
for index, row in city_df.iterrows():

    # get nearest hotel type from df
    hotel_type = row['city']

    # add keyword to params dict
    params['keyword'] = hotel_type

# assemble url and make API request
response = requests.get(base_url, params=params).json()
# extract results
results = response['results']
    
try:
        print(f"Closest {hotel_type} hotel is {results[0]['name']}.")
        
except (KeyError, IndexError):
            print("Sorry,hotel not found")

NameError: name 'g_key' is not defined

In [17]:
hotel_df

,City,Country,Lat,Lon,Hotel Name
0,Sobolevo,NaN,54.4299,31.8996,
1,Punta Arenas,NaN,-53.1500,-70.9167,
2,Albany,NaN,42.6001,-73.9662,
3,Baykit,NaN,61.6700,96.3700,
4,Saskylakh,NaN,71.9167,114.0833,
...,...,...,...,...,...
553,Tadpatri,NaN,14.9167,78.0167,
554,Hambantota,NaN,6.1241,81.1185,
555,Saint Anthony,NaN,45.0205,-93.2180,
556,Mossendjo,NaN,-2.9506,12.7261,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig



Figure(layout=FigureLayout(height='420px'))